In [1]:
# PHUSE CSS Hackaton 2020
# Nicolas Dupuis - Geoffrey Low
# Purpose: create an ADSL dataset

In [ ]:
import python.podr_connections as podr
import pandas as pd
import subprocess
import yaml, json

# first time
#subprocess.run(["pip3", "install", 'xlrd'])

## Connect to PODR

In [2]:
connection = podr.podr_connection(username='phuse_3i3892wcjd')
# password: GpAMqVCDMPxo

PODR Password: ········


## Fetch SDTM datasets

In [3]:
dm = connection.read(dataset = 'dm', libname = 'cdisc_pilot_sdtm')
dm.head()

,STUDYID,DOMAIN,USUBJID,SUBJID,RFSTDTC,RFENDTC,RFXSTDTC,RFXENDTC,RFICDTC,RFPENDTC,...,SEX,RACE,ETHNIC,ARMCD,ARM,ACTARMCD,ACTARM,COUNTRY,DMDTC,DMDY
0,CDISCPILOT01,DM,01-701-1015,1015,2014-01-02,2014-07-02,2014-01-02,2014-07-02,None,2014-07-02T11:45,...,F,WHITE,HISPANIC OR LATINO,Pbo,Placebo,Pbo,Placebo,USA,2013-12-26,-7.0
1,CDISCPILOT01,DM,01-701-1023,1023,2012-08-05,2012-09-02,2012-08-05,2012-09-01,None,2013-02-18,...,M,WHITE,HISPANIC OR LATINO,Pbo,Placebo,Pbo,Placebo,USA,2012-07-22,-14.0
2,CDISCPILOT01,DM,01-701-1028,1028,2013-07-19,2014-01-14,2013-07-19,2014-01-14,None,2014-01-14T11:10,...,M,WHITE,NOT HISPANIC OR LATINO,Xan_Hi,Xanomeline High Dose,Xan_Hi,Xanomeline High Dose,USA,2013-07-11,-8.0
3,CDISCPILOT01,DM,01-701-1033,1033,2014-03-18,2014-04-14,2014-03-18,2014-03-31,None,2014-09-15,...,M,WHITE,NOT HISPANIC OR LATINO,Xan_Lo,Xanomeline Low Dose,Xan_Lo,Xanomeline Low Dose,USA,2014-03-10,-8.0
4,CDISCPILOT01,DM,01-701-1034,1034,2014-07-01,2014-12-30,2014-07-01,2014-12-30,None,2014-12-30T09:50,...,F,WHITE,NOT HISPANIC OR LATINO,Xan_Hi,Xanomeline High Dose,Xan_Hi,Xanomeline High Dose,USA,2014-06-24,-7.0


In [4]:
ds = connection.read(dataset = 'ds', libname = 'cdisc_pilot_sdtm')
ds.head()

,STUDYID,DOMAIN,USUBJID,DSSEQ,DSSPID,DSTERM,DSDECOD,DSCAT,VISITNUM,VISIT,EPOCH,DSDTC,DSSTDTC,DSDY,DSSTDY
0,CDISCPILOT01,DS,01-701-1015,1.0,NaN,PROTOCOL COMPLETED,COMPLETED,DISPOSITION EVENT,13.0,WEEK 26,FOLLOW-UP,2014-07-02,2014-07-02,182.0,182.0
1,CDISCPILOT01,DS,01-701-1015,2.0,NaN,FINAL LAB VISIT,FINAL LAB VISIT,OTHER EVENT,13.0,WEEK 26,FOLLOW-UP,2014-07-02T11:45,2014-07-02,182.0,182.0
2,CDISCPILOT01,DS,01-701-1023,1.0,24.0,ADVERSE EVENT,ADVERSE EVENT,DISPOSITION EVENT,5.0,WEEK 4,TREATMENT,2012-09-02,2012-09-02,29.0,29.0
3,CDISCPILOT01,DS,01-701-1023,2.0,NaN,FINAL LAB VISIT,FINAL LAB VISIT,OTHER EVENT,5.0,WEEK 4,TREATMENT,2012-09-02T10:15,2012-09-02,29.0,29.0
4,CDISCPILOT01,DS,01-701-1023,3.0,NaN,FINAL RETRIEVAL VISIT,FINAL RETRIEVAL VISIT,OTHER EVENT,201.0,RETRIEVAL,FOLLOW-UP,2013-02-18,2013-02-18,198.0,198.0


## Fetch ADaM Metadata

In [5]:
adam_metadata = pd.read_excel('PHUSE CSS_2020_hackathon_AD usecase.xlsx', sheet_name='ADSL')
adsl_metadata = adam_metadata [adam_metadata ['Dataset Name']=='ADSL']
adsl_metadata.head()

,Dataset Name,Variable Name,Variable Label,Variable Role,Variable Type,Parameter Identifier,Variable Order,Length,Dec Digits,Display Format,Codelist/Controlled Terms,Core,Source/Derivation,Derived or Copied from Source Variable
0,ADSL,STUDYID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Required,DM.STUDYID,Copied from Source
1,ADSL,USUBJID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Required,DM.USUBJID,Copied from Source
2,ADSL,ARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Required,DM.ARM,Copied from Source
3,ADSL,ARMCD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Permissible,DM.ARMCD,Copied from Source
4,ADSL,ACTARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Permissible,DM.ACTARM,Copied from Source


## Fetch ADSL derivations

In [34]:
with open('derivations.json', 'r') as f:
    derivations = json.load(f)
derivations

{'AGE_RACE': {'broadcast': ['SEX', 'RACE']}}

## Main Class

In [39]:
class create_adam():
    
    def __init__(self, metadata, source):
        
        self.source = source.copy()
        
        # variable that should be copied from source
        vars_to_copy = set(metadata [metadata ['Derived or Copied from Source Variable']=='Copied from Source']['Variable Name'])
        
        # variables from source
        source_variables = set(source.columns)
        
        # variables that should and can be copied from source
        self.copy_variables = vars_to_copy.intersection(source_variables)
        
        # variables that should be copied but cannot
        not_in_source = vars_to_copy.difference(source_variables)
        if len(not_in_source) >0: 
            print(f"The following variables are not available in source: {not_in_source}")           

    
    def source_variables(self):
        ''' Filter the SDTM source dataset to keep only the 'copied_from_source' variables '''
        self.adsl = self.source[self.copy_variables]
        self.adsl = self.adsl.copy() # avoid the ugly warning
        return self.adsl
    

    def create_newvars(self, row):

        for derive_var in self.derivations:
            
            # Code should create a 'value' variable for the new derived variable
            code = self.derivations[derive_var]['derivation']
            exec(code) 

            # Store the value into the dict
            self.derivations[derive_var]['value'] = value

        # Return all new variables name/value
        return {'RANDFL':self.derivations[derive_var]['value'] }
        #return {var:value for var,value in derive_vars}
    
    
    def derived_variables(self, derivations):

        self.derivations = derivations.copy()
        
        for variable in self.derivations:
            
            specs = self.derivations[variable]
            
            if 'category' in specs:
                
                source = specs['category']['source']
                bins = specs['category']['bins']
                self.adsl[variable] = pd.cut(self.adsl[source], bins)

            elif 'broadcast' in specs:        
                
                items = ["self.adsl['" + item +"']" for item in specs['broadcast']]
                concat = ' + "-" + '.join(items)
                self.adsl[variable] = eval(concat)
                
            elif 'code' in self.derivations[variable]:
        
                df_newvars = self.adsl.apply(self.create_newvars, axis='columns', result_type='expand')

                self.adsl = pd.concat([self.adsl, df_newvars], axis='columns')
        
        return adsl

# Instantiate class
create_adsl = create_adam(metadata=adsl_metadata, source=dm)

# Add variables copied from source
adsl = create_adsl.source_variables() 

# Add derived variables
adsl = create_adsl.derived_variables(derivations)

The following variables are not available in source: {'INVNAM'}
["self.adsl['SEX']", "self.adsl['RACE']"]
self.adsl['SEX'] + "-" + self.adsl['RACE']


In [40]:
adsl.head()

,ACTARM,COUNTRY,ACTARMCD,STUDYID,AGE,SUBJID,AGEU,USUBJID,SEX,RACE,ETHNIC,ARM,ARMCD,DTHDTC,DTHFL,SITEID,AGE_RACE
0,Placebo,USA,Pbo,CDISCPILOT01,63.0,1015,YEARS,01-701-1015,F,WHITE,HISPANIC OR LATINO,Placebo,Pbo,None,None,701,F-WHITE
1,Placebo,USA,Pbo,CDISCPILOT01,64.0,1023,YEARS,01-701-1023,M,WHITE,HISPANIC OR LATINO,Placebo,Pbo,None,None,701,M-WHITE
2,Xanomeline High Dose,USA,Xan_Hi,CDISCPILOT01,71.0,1028,YEARS,01-701-1028,M,WHITE,NOT HISPANIC OR LATINO,Xanomeline High Dose,Xan_Hi,None,None,701,M-WHITE
3,Xanomeline Low Dose,USA,Xan_Lo,CDISCPILOT01,74.0,1033,YEARS,01-701-1033,M,WHITE,NOT HISPANIC OR LATINO,Xanomeline Low Dose,Xan_Lo,None,None,701,M-WHITE
4,Xanomeline High Dose,USA,Xan_Hi,CDISCPILOT01,77.0,1034,YEARS,01-701-1034,F,WHITE,NOT HISPANIC OR LATINO,Xanomeline High Dose,Xan_Hi,None,None,701,F-WHITE
